In [3]:
import pandas as pd
import os
import csv
import time
import dateutil.parser as dparser
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
from googleapiclient import discovery
from datetime import datetime, timedelta

#Read Bionic CSV export
def get_csv():
    df = pd.read_csv('Beamly - All Placements.csv', encoding = "ISO-8859-1",mangle_dupe_cols=True, usecols=['Campaign ID','Campaign Name','Line Type','Channel.1', 'Parent Advertiser Name','Start Date','End Date','Agency Comp','Total Media Cost Currency','Total Media Cost','Last Updated By','Bid Type','Creative Size','Device Type','Format Type','Inventory Type','Objective','Rate Type','Site','Targeting Sub-Type','Targetting Type', 'Line ID'], parse_dates=['Start Date', 'End Date'])
    df = df[df['Line Type'].str.contains('Placement')]
    df = df[df['Campaign ID'] > 65234]
    df =  df.fillna('EMPTYFIELD')

    return df

def main():
    df = get_csv()

    varA      = 'Campaign ID';
    dfGrouped = df.groupby(varA, as_index=False).agg({'Start Date': 'min', 'End Date': 'max'}).copy()

    varsToKeep = ['Campaign ID', 'Campaign Name', 'Site', 'Inventory Type', 'Bid Type', 'Creative Size', 'Targetting Type', 'Device Type','Targeting Sub-Type', 'Rate Type', 'Objective', 'Channel.1', 'Format Type', 'Start Date_grp', 'End Date_grp', 'Total Media Cost', 'Line ID' ]
    dfTemp = pd.merge(df, dfGrouped, how='inner', on='Campaign ID', suffixes=(' ', '_grp'), copy=True)[varsToKeep]

 
    dfTemp.to_csv(path_or_buf='dfTemp.csv')

    #Rename column names to avoid spaces
    dfTemp['Rate_Type'] = dfTemp['Rate Type']
    dfTemp['Format_Type'] = dfTemp['Format Type']
    dfTemp['Channel'] = dfTemp['Channel.1']
    dfTemp['Creative_Size'] = dfTemp['Creative Size']
    dfTemp['Bid_Type'] = dfTemp['Bid Type']
    dfTemp['Targetting_Type'] = dfTemp['Targetting Type']
    dfTemp['Device_Type'] = dfTemp['Device Type']
    dfTemp['Targeting_Sub_Type'] = dfTemp['Targeting Sub-Type']
    dfTemp['Start_Date'] = dfTemp['Start Date_grp']
    dfTemp['End_Date'] = dfTemp['End Date_grp']
    dfTemp['Inventory_Type'] = dfTemp['Inventory Type']
    dfTemp['Line_ID'] = dfTemp['Line ID']


    #Checks if there are empty fields
    for row in dfTemp.itertuples():

        #Site
        if row[3] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Site == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Site'] = 'EMPTYFIELD'
        #Inventory Type
        if row[4] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Inventory_Type == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Inventory_Type'] = 'EMPTYFIELD'
        #Bid Type 
        if row[5] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Bid_Type == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Bid_Type'] = 'EMPTYFIELD'
        #Creative Size 
        if row[6] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Creative_Size == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Creative_Size'] = 'EMPTYFIELD'
        #Targetting Type 
        if row[7] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Creative_Size == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Creative_Size'] = 'EMPTYFIELD'
        #Device Type 
        if row[8] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Device_Type == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Device_Type'] = 'EMPTYFIELD'
        #Targeting Sub-Type 
        if row[9] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Targeting_Sub_Type == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Targeting_Sub_Type'] = 'EMPTYFIELD'
        #Rate 
        if row[10] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Rate_Type == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Rate_Type'] = 'EMPTYFIELD'
        #Objective 
        if row[11] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Objective == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Objective'] = 'EMPTYFIELD'
        #Channel 
        if row[12] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Channel == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Channel'] = 'EMPTYFIELD'
        #Format Type 
        if row[13] == "EMPTYFIELD":
            ids = dfTemp['Campaign ID'][dfTemp.Format_Type == 'EMPTYFIELD'].unique()
            dfTemp.loc[dfTemp['Campaign ID'].isin(ids), 'Format_Type'] = 'EMPTYFIELD'

 

    '''
    Bionic_error = dfTemp[dfTemp.Site.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Inventory_Type.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Bid_Type.str.contains('EMPTYFIELD')]

    '''

    Bionic_error = dfTemp[dfTemp.Site.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Inventory_Type.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Bid_Type.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Creative_Size.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Targetting_Type.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Device_Type.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Targeting_Sub_Type.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Rate_Type.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Objective.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Channel.str.contains('EMPTYFIELD')]
    Bionic_error = Bionic_error[Bionic_error.Format_Type.str.contains('EMPTYFIELD')]

    Bionic_good = dfTemp[~dfTemp.Site.str.contains('EMPTYFIELD')]
    Bionic_good = Bionic_good[~Bionic_good.Inventory_Type.str.contains('EMPTYFIELD')]
    Bionic_good = Bionic_good[~Bionic_good.Bid_Type.str.contains('EMPTYFIELD')]
    Bionic_good = Bionic_good[~Bionic_good.Creative_Size.str.contains('EMPTYFIELD')]
    Bionic_good = Bionic_good[~Bionic_good.Targetting_Type.str.contains('EMPTYFIELD')]
    Bionic_good = Bionic_good[~Bionic_good.Device_Type.str.contains('EMPTYFIELD')]
    Bionic_good = Bionic_good[~Bionic_good.Targeting_Sub_Type.str.contains('EMPTYFIELD')]
    Bionic_good = Bionic_good[~Bionic_good.Rate_Type.str.contains('EMPTYFIELD')]
    Bionic_good = Bionic_good[~Bionic_good.Objective.str.contains('EMPTYFIELD')]
    Bionic_good = Bionic_good[~Bionic_good.Channel.str.contains('EMPTYFIELD')]
    Bionic_good = Bionic_good[~Bionic_good.Format_Type.str.contains('EMPTYFIELD')]

 

    Bionic_error = Bionic_error.groupby(['Campaign ID', 'Campaign Name', 'Site', 'Inventory_Type', 'Bid_Type', 'Creative_Size', 'Targetting_Type', 'Device_Type','Targeting_Sub_Type', 'Rate_Type', 'Objective', 'Channel', 'Format_Type', 'Start_Date', 'End_Date', 'Line ID']).agg({'Start Date_grp': 'min', 'End Date_grp': 'max', 'Total Media Cost': 'sum'})
    Bionic_good = Bionic_good.groupby(['Campaign ID', 'Campaign Name', 'Site', 'Inventory_Type', 'Bid_Type', 'Creative_Size', 'Targetting_Type', 'Device_Type','Targeting_Sub_Type', 'Rate_Type', 'Objective', 'Channel', 'Format_Type', 'Start_Date', 'End_Date', 'Line ID']).agg({'Start Date_grp': 'min', 'End Date_grp': 'max', 'Total Media Cost': 'sum'})
    

    Bionic_error.to_csv(path_or_buf='Bionic_error.csv', sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, date_format=None, doublequote=True, escapechar=None, decimal='.')
    Bionic_good.to_csv(path_or_buf='Bionic_good.csv', sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, date_format=None, doublequote=True, escapechar=None, decimal='.')
    

    return

if __name__ == "__main__":
        #Main
        main()